In [ ]:
from twarc import Twarc
import os

import pandas as pd
import tweepy
from multiprocessing import Pool
from pymongo import MongoClient
from collections.abc import Iterable

TWITTER_CONSUMER_KEY = "nCfs9fJCXThG9PHVWinsVywHt"
TWITTER_CONSUMER_SECRET = "EvFpByStx6diPJ0UZplSkhBd53laQj5lheE0KynGojEtTtAy17"
TWITTER_ACCESS_TOKEN = "1380480442818789376-xXSTw2pNEzHUgJ486YFkxPeXXkWSGX"
TWITTER_ACCESS_TOKEN_SECRET = "R7sSTfgCV3O2ZZryWtRNQZHTc8blOJN60zqhM46FG6ak9"


# path of file that contains a list of tweet IDs
file_name = os.path.join(os.getcwd(), "IEEE_covid_tweets.csv")

In [ ]:
mongo_client = MongoClient()
mongo_db = mongo_client["twitter"]
mongo_collection = mongo_db["tweets"]


# check existence of tweet in MongoDB
def is_tweet_existing(collection, tweet_id):
    try:
        resultSet = collection.find({'id_str':str(tweet_id)})
        if resultSet.count() > 0:
            if isinstance(resultSet, Iterable):
                return True
            else:
                return True
        else:
            return False
    except Exception as e:
        print('Exception : %s' % str(e))

#save to MongoDB
def insert_into_mongo(collection, tweet): 
    try:
        print("New tweet will be stored >> ", tweet['id_str'])
        print("I will insert it!")
        collection.insert(tweet)
        return True
    except Exception as e:
        print('Exception : %s' % str(e))

In [ ]:
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


lines = []
with open(file_name) as f:
    lines = f.readlines()
print("Reading tweets IDs file is done!")

tweet_ids = []
for line in lines:
    tweet_ids.append(line.replace("\n", "").split(",")[0].strip())

print("There are {} tweets IDs!".format(len(tweet_ids)))


def _worker_scrape_tweet(tweet_id):
    try:
        tweet = api.get_status(int(tweet_id))._json
        if "id" in tweet.keys():
            print(tweet['id'])
        insert_into_mongo(mongo_collection, tweet)
        return tweet
    except Exception as e:
        print("Exception:", e)
        pass

Remove tweets ids that already has been scrapped

In [ ]:
tweet_ids = set(tweet_ids) - set(pd.DataFrame(mongo_collection.distinct("id_str"), columns=["id_str"])['id_str'].tolist())
len(tweet_ids)

In [ ]:
# tweets = []

# threads_num = 5
# pool = Pool(threads_num)
# tweets = pool.map(_worker_scrape_tweet, tweet_ids)
# pool.close()

In [ ]:
tweets = []
for tweet_id in tweet_ids:
#     if is_tweet_existing(mongo_collection, tweet_id):
#         print(">>> Tweet is already stored >> ", tweet_id)
#     else:
    tweets.append(_worker_scrape_tweet(tweet_id))

In [ ]:
print("Number of scraped tweets is {}".format(len(tweets)))

In [ ]:
tweets[0]